# Feature Engineering and Modelling

### <u>The Physics & Economics of Price: Why Temperature is Non-Linear</u>

`temp_c` is arguably the most critical external factor for predicting `price_actual`. However, our correlation matrix showed a low linear correlation ($\approx 0.11$). This suggests the relationship is **non-linear**, and using a standard linear model without transformation would result in significant under-fitting.

**<u>The Core Axiom: The Causal Chain</u>**

To understand why the data looks this way, we must look at the fundamental chain of causality in energy markets:

> **Weather influences Human Behavior $\rightarrow$ Human Behavior drives Demand $\rightarrow$ Demand drives Price.**

The grid does not care about temperature; it cares about how **humans react** to temperature. This reaction is not uniform, it depends entirely on the season.

**<u>Analysis of the "U-Shaped" Relationship</u>**

We can categorize this human behavior into three distinct thermal regimes:

1.  **The Summer Regime (Cooling Load):**
    During hot weather ($> 25^\circ C$), human behavior shifts towards seeking comfort. People turn on Air Conditioning. This creates a massive surge in **Demand**, which instantly drives **Price** up.
    *   *Mathematical Behavior:* Positive Linear Relationship ($Slope > 0$).

2.  **The Winter Regime (Heating Load):**
    During freezing weather ($< 5^\circ C$), human behavior shifts towards survival. People maximize electric heating systems. This spike in **Demand** forces expensive peaker plants online, driving **Price** up.
    *   *Mathematical Behavior:* Inverse Linear Relationship ($Slope < 0$).

3.  **The "Comfort Zone" (Shoulder Months):**
    During mild weather ($15^\circ C - 20^\circ C$), human behavior becomes passive. Windows are opened, and neither AC nor Heating is required. **Demand** collapses to its minimum baseload, resulting in the lowest **Prices** of the year.
    *   *Mathematical Behavior:* The Global Minimum of the curve.

**<u>The Solution: Polynomial Transformation</u>**

Since the relationship goes "Down" (Winter), "Flattens" (Spring), and then "Up" (Summer), it mathematically forms a **Parabola** ($y = x^2$).

To capture this, we cannot simply use `temp_c`. We must apply a **Polynomial Transformation**. This achieves two goals:
1.  **Curvature:** By introducing `temp_squared`, the model can fit the "U-Shape" described above.
2.  **Interaction:** By using `PolynomialFeatures`, we also capture complex weather interactions (e.g., `temp_c * solar_radiation`), allowing the model to understand scenarios like *"It is hot (High Demand), but the Sun is shining (High Supply)."*

In [18]:
# Imports and Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# Preprocessing and Pipelines
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Models 
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor

# Validation and Metrics
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, TimeSeriesSplit

In [19]:
df = pd.read_csv("../data/processed/trained_data.csv")
display(df.head())

,datetime_beginning_ept,price_actual,hour_of_day,day_of_week,month,price_1h_ago,price_24h_ago,avg_price_last_24h,temp_c,wind_kph,solar_radiation
0,2024-01-02 00:00:00,21.3249,0.0,2.0,1.0,23.3484,31.3827,29.3789,2.0,10.8,0.0
1,2024-01-02 01:00:00,19.6885,1.0,2.0,1.0,21.3249,20.0838,28.9598,1.2,8.0,0.0
2,2024-01-02 02:00:00,20.0916,2.0,2.0,1.0,19.6885,17.6052,28.9433,0.4,6.4,0.0
3,2024-01-02 03:00:00,18.6212,3.0,2.0,1.0,20.0916,19.7673,29.0469,0.5,10.7,0.0
4,2024-01-02 04:00:00,18.6391,4.0,2.0,1.0,18.6212,17.0687,28.9992,-0.3,12.0,0.0


In [20]:
# Converting the datetime column to datetime type
df["datetime_beginning_ept"] = pd.to_datetime(df["datetime_beginning_ept"])

# Setting back the datetime index
df = df.set_index("datetime_beginning_ept")
df.sort_index(inplace=True)
display(df.head())

,price_actual,hour_of_day,day_of_week,month,price_1h_ago,price_24h_ago,avg_price_last_24h,temp_c,wind_kph,solar_radiation
datetime_beginning_ept,,,,,,,,,,
2024-01-02 00:00:00,21.3249,0.0,2.0,1.0,23.3484,31.3827,29.3789,2.0,10.8,0.0
2024-01-02 01:00:00,19.6885,1.0,2.0,1.0,21.3249,20.0838,28.9598,1.2,8.0,0.0
2024-01-02 02:00:00,20.0916,2.0,2.0,1.0,19.6885,17.6052,28.9433,0.4,6.4,0.0
2024-01-02 03:00:00,18.6212,3.0,2.0,1.0,20.0916,19.7673,29.0469,0.5,10.7,0.0
2024-01-02 04:00:00,18.6391,4.0,2.0,1.0,18.6212,17.0687,28.9992,-0.3,12.0,0.0


In [21]:
# Verifying the index
df.index

DatetimeIndex(['2024-01-02 00:00:00', '2024-01-02 01:00:00',
               '2024-01-02 02:00:00', '2024-01-02 03:00:00',
               '2024-01-02 04:00:00', '2024-01-02 05:00:00',
               '2024-01-02 06:00:00', '2024-01-02 07:00:00',
               '2024-01-02 08:00:00', '2024-01-02 09:00:00',
               ...
               '2024-12-30 15:00:00', '2024-12-30 16:00:00',
               '2024-12-30 17:00:00', '2024-12-30 18:00:00',
               '2024-12-30 19:00:00', '2024-12-30 20:00:00',
               '2024-12-30 21:00:00', '2024-12-30 22:00:00',
               '2024-12-30 23:00:00', '2024-12-31 00:00:00'],
              dtype='datetime64[ns]', name='datetime_beginning_ept', length=8736, freq=None)

In [22]:
# Summary Function 
def summarize_df(df, df_name="df"):
    """
    Display key information about a DataFrame:
    - info()
    - describe()
    - duplicated rows
    - count of missing values
    """
    print(f"===== DataFrame ({df_name.upper()}) Summary =====")
    print("===== DataFrame Index =====")
    display(df.index)
    print("===== DataFrame Info =====")
    df.info()
    print("\n===== DataFrame Description =====")
    display(df.describe(include='all'))  # include='all' to describe non-numeric columns too
    print("\n===== Duplicate Rows =====")
    duplicates = df[df.duplicated(keep=False)]
    if not duplicates.empty:
        display(duplicates)
    else:
        print("No duplicate rows found.")
    print("\n===== Missing Values per Column =====")
    print(df.isna().sum())


summarize_df(df, "train_df")

===== DataFrame (TRAIN_DF) Summary =====
===== DataFrame Index =====


DatetimeIndex(['2024-01-02 00:00:00', '2024-01-02 01:00:00',
               '2024-01-02 02:00:00', '2024-01-02 03:00:00',
               '2024-01-02 04:00:00', '2024-01-02 05:00:00',
               '2024-01-02 06:00:00', '2024-01-02 07:00:00',
               '2024-01-02 08:00:00', '2024-01-02 09:00:00',
               ...
               '2024-12-30 15:00:00', '2024-12-30 16:00:00',
               '2024-12-30 17:00:00', '2024-12-30 18:00:00',
               '2024-12-30 19:00:00', '2024-12-30 20:00:00',
               '2024-12-30 21:00:00', '2024-12-30 22:00:00',
               '2024-12-30 23:00:00', '2024-12-31 00:00:00'],
              dtype='datetime64[ns]', name='datetime_beginning_ept', length=8736, freq=None)

===== DataFrame Info =====
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8736 entries, 2024-01-02 00:00:00 to 2024-12-31 00:00:00
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   price_actual        8736 non-null   float64
 1   hour_of_day         8736 non-null   float64
 2   day_of_week         8736 non-null   float64
 3   month               8736 non-null   float64
 4   price_1h_ago        8736 non-null   float64
 5   price_24h_ago       8736 non-null   float64
 6   avg_price_last_24h  8736 non-null   float64
 7   temp_c              8736 non-null   float64
 8   wind_kph            8736 non-null   float64
 9   solar_radiation     8736 non-null   float64
dtypes: float64(10)
memory usage: 750.8 KB

===== DataFrame Description =====


,price_actual,hour_of_day,day_of_week,month,price_1h_ago,price_24h_ago,avg_price_last_24h,temp_c,wind_kph,solar_radiation
count,8736.000000,8736.000000,8736.000000,8736.000000,8736.000000,8736.000000,8736.000000,8736.000000,8736.000000,8736.000000
mean,33.278590,11.499771,3.000229,6.514766,33.280508,33.313805,33.300629,13.217903,9.870650,173.569254
std,28.297715,6.922930,1.999886,3.437097,28.297486,28.294552,14.345797,9.908276,5.366993,250.165110
min,-34.773600,0.000000,0.000000,1.000000,-34.773600,-34.773600,11.098400,-11.400000,0.200000,0.000000
25%,19.222125,5.750000,1.000000,4.000000,19.222125,19.270475,24.265900,5.200000,6.100000,0.000000
50%,26.266850,11.500000,3.000000,7.000000,26.266850,26.314500,30.366700,13.650000,8.700000,8.000000
75%,37.270625,17.250000,5.000000,10.000000,37.283700,37.302700,37.931675,21.000000,12.600000,307.000000
max,492.583300,23.000000,6.000000,12.000000,492.583300,492.583300,117.905700,37.000000,36.800000,980.000000



===== Duplicate Rows =====
No duplicate rows found.

===== Missing Values per Column =====
price_actual          0
hour_of_day           0
day_of_week           0
month                 0
price_1h_ago          0
price_24h_ago         0
avg_price_last_24h    0
temp_c                0
wind_kph              0
solar_radiation       0
dtype: int64


## Training and Evaluating Multiple Regression Models

In [23]:
# Since we are using ColumnTransformer, we need to have the columns' names available
X = df.drop(columns=["price_actual"])
y = df["price_actual"]

In [24]:
display(df.head())

,price_actual,hour_of_day,day_of_week,month,price_1h_ago,price_24h_ago,avg_price_last_24h,temp_c,wind_kph,solar_radiation
datetime_beginning_ept,,,,,,,,,,
2024-01-02 00:00:00,21.3249,0.0,2.0,1.0,23.3484,31.3827,29.3789,2.0,10.8,0.0
2024-01-02 01:00:00,19.6885,1.0,2.0,1.0,21.3249,20.0838,28.9598,1.2,8.0,0.0
2024-01-02 02:00:00,20.0916,2.0,2.0,1.0,19.6885,17.6052,28.9433,0.4,6.4,0.0
2024-01-02 03:00:00,18.6212,3.0,2.0,1.0,20.0916,19.7673,29.0469,0.5,10.7,0.0
2024-01-02 04:00:00,18.6391,4.0,2.0,1.0,18.6212,17.0687,28.9992,-0.3,12.0,0.0


In [25]:
"""MODIFICATIONS TO MADE: 
- Calcualte a better n_splits for TimeSeriesSplit based on n_splits ≈ (N / H) - 1. Needs to understand the procedure
"""


weather_cols: list[str] = ["temp_c", "wind_kph", "solar_radiation"]
other_cols: list[str] = ["price_1h_ago", "price_24h_ago", "avg_price_last_24h", "hour_of_day", "day_of_week", "month"]



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42, shuffle=False)

# Used for Time Series based Cross-Validation 
tscv = TimeSeriesSplit(n_splits=30, test_size=24) # Forecast horizon of 24 hours over a period of 30 days (720 hours)

"""Pipeline for weather columns"""
weather_poly_scaled = Pipeline(steps=[
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", RobustScaler())
])

"""Pipeline for other columns"""
other_scaled = Pipeline(steps=[
    ("scaler", RobustScaler())
])

"""Processor for Linear and Non-Distance model"""
processor_scaled = ColumnTransformer(
    transformers=[
        ("weather_columns", weather_poly_scaled, weather_cols),
        ("other_columns", other_scaled, other_cols)
    ],
    remainder="drop"
)

"""Processor for Tree-model"""
processor_trees = ColumnTransformer(
    transformers=[
        ("all_columns", "passthrough", weather_cols + other_cols)
    ],
    remainder="drop"
)

# Dict of all pipelines
pipelines: dict = {
    "LinearRegression": Pipeline(steps=[
        ("process", processor_scaled),
        ("lr", LinearRegression())
    ]),

    "RidgeRegression": Pipeline(steps=[
        ("process", processor_scaled),
        ("ridge", Ridge())
    ]),

    "LassoRegression": Pipeline(steps=[
        ("process", processor_scaled),
        ("lasso", Lasso())
    ]),

    "KNN": Pipeline(steps=[
        ("process", processor_scaled),
        ("knn", KNeighborsRegressor())
    ]),

    "BaggingRegressor": Pipeline(steps=[
        ("process", processor_trees),
        ("br", BaggingRegressor(random_state=42, verbose=1))
    ]),

    "RandomForestRegressor": Pipeline(steps=[
        ("process", processor_trees),
        ("rfr", RandomForestRegressor(random_state=42, verbose=1))
    ]),

    "GradientBoostingRegressor": Pipeline(steps=[
        ("process", processor_trees),
        ("gbr", GradientBoostingRegressor(random_state=42, verbose=1))
    ])
}

# Hyperparameter grids for each model
param_grid: dict = {
    "Ridge": {"alpha": [0.01, 0.1, 1.0, 10.0]},
    "Lasso": {"alpha": [0.001, 0.01, 0.1, 1.0]},
    "KNN": {
        "n_neighbors": [3, 5, 7, 9],
        "weights": ["uniform", "distance"]
    },
    "RandomForestRegressor": {
        "n_estimators": [100, 200],
        "max_depth": [None, 10, 20],
        "max_features": ["sqrt", "log2"]
    },
    "BaggingRegressor": {
        "n_estimators": [10, 50, 100], # Uses Decision Tree as base estimator by default
        "max_samples": [0.6, 0.8, 1.0]
    },
    "GradientBoostingRegressor": {
        "n_estimators": [100, 200],
        "learning_rate": [0.05, 0.1],
        "max_depth": [2, 3, 4]
    },
}

# CROSS VALIDATION SETUP
results: list = []  #Stores the CV_Scores for each model
estimators: dict = {}
cv_score: float = 0.0
best_score: float = 0.0

print(f"Starting Training loop on {len(pipelines)} models...")
print("-" * 50)

for name, pipeline in pipelines.items():
    print(f"Training (Fitting) {name}")

    # Pipeline uses it to go through all parameters
    # pipelien__parameter: values
    search_params: dict = {} 

    step_name: str = "" # To prevent UnboundError

    # Identify which algorithm to it is 
    if "RidgeRegression" in name: grid_key, step_name = "Ridge", "ridge"
    elif "LassoRegression" in name: grid_key, step_name = "Lasso", "lasso"
    elif "KNN" in name: grid_key, step_name = "KNN", "knn"
    elif "RandomForestRegressor" in name: grid_key, step_name = "RandomForestRegressor", "rfr"
    elif "BaggingRegressor" in name: grid_key, step_name = "BaggingRegressor", "br"
    elif "GradientBoostingRegressor" in name: grid_key, step_name = "GradientBoostingRegressor", "gbr"
    else: grid_key = None  # No hyperparameter tuning for Linear Regression


    # Build the parameter grid for GridSearchCV
    if grid_key and grid_key in param_grid:
        for param, values in param_grid[grid_key].items():
            search_params[f"{step_name}__{param}"] = values

    # Run GridSearchCV if the model is in the param_grid
    if search_params: 
        model = GridSearchCV(
            pipeline, 
            param_grid=search_params, 
            cv=tscv, 
            scoring="neg_root_mean_squared_error", 
            n_jobs=-1, 
            verbose=1
        )

        # Fitting the model on the entire training data
        model.fit(X_train, y_train)
        final_model = model.best_estimator_ # Best pipeline with best hyperparameters
        best_score = round(-model.best_score_, 4) # Best CV score (lowest RMSE)
        best_params = model.best_params_ # Best hyperparameters

        # Saving the CV results for analysis
        cv_results = pd.DataFrame(model.cv_results_)
        display(cv_results.sort_values("mean_test_score"))
        display(cv_results[["params","mean_test_score","std_test_score"]].sort_values("mean_test_score"))


    else: # If the model was not in the param_grid, fit the pipeline directly
        cv_scores = cross_val_score(
            pipeline,
            X_train, 
            y_train, 
            cv=tscv, 
            scoring="neg_root_mean_squared_error"
         )

        cv_score = round(-cv_scores.mean(), 4) # AVG_RMSE 
        best_params = "Default"

        # Fitting the model on the entire training data
        pipeline.fit(X_train, y_train)
        final_model = pipeline

    
    # Store results
    results.append({
        "Model": name,
        "CV_RMSE": best_score if search_params else cv_score,
        "Best Params": best_params
    })

    # Saving the model object
    estimators[name] = final_model 
    print(f"Completed training for {name}. CV_RMSE: {results[-1]["CV_RMSE"]:.4f}")


# LEADERBOARD
print("-" * 50)
print("Printing the Model Leaderboard sorted by CV_RMSE:")
leaderb_df = pd.DataFrame(results).sort_values("CV_RMSE", ascending=True)
display(leaderb_df)

# SAVING MODELS 
print("Saving models to disk...")
joblib.dump(estimators, "../data/processed/estimators.pkl") 
print("Done.")

Starting Training loop on 7 models...
--------------------------------------------------
Training (Fitting) LinearRegression
Completed training for LinearRegression. CV_RMSE: 20.4526
Training (Fitting) RidgeRegression
Fitting 30 folds for each of 4 candidates, totalling 120 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_ridge__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,split23_test_score,split24_test_score,split25_test_score,split26_test_score,split27_test_score,split28_test_score,split29_test_score,mean_test_score,std_test_score,rank_test_score
3,0.053050,0.008755,0.015062,0.004526,10.00,{'ridge__alpha': 10.0},-15.552225,-15.178155,-6.926497,-12.664454,...,-5.489208,-9.041293,-30.818927,-37.359318,-40.283687,-25.794928,-36.462000,-20.457261,11.239035,4
2,0.057139,0.013826,0.019931,0.016934,1.00,{'ridge__alpha': 1.0},-15.541202,-15.177931,-6.955158,-12.678994,...,-5.506847,-8.956096,-30.790264,-37.317193,-40.247895,-25.773216,-36.471731,-20.453117,11.243023,3
1,0.059651,0.018133,0.017610,0.007694,0.10,{'ridge__alpha': 0.1},-15.540114,-15.178020,-6.958593,-12.680744,...,-5.508878,-8.946384,-30.786942,-37.312410,-40.243898,-25.770690,-36.472903,-20.452657,11.243485,2
0,0.064066,0.017138,0.019677,0.009128,0.01,{'ridge__alpha': 0.01},-15.540005,-15.178030,-6.958943,-12.680923,...,-5.509084,-8.945399,-30.786604,-37.311925,-40.243493,-25.770434,-36.473022,-20.452610,11.243532,1


,params,mean_test_score,std_test_score
3,{'ridge__alpha': 10.0},-20.457261,11.239035
2,{'ridge__alpha': 1.0},-20.453117,11.243023
1,{'ridge__alpha': 0.1},-20.452657,11.243485
0,{'ridge__alpha': 0.01},-20.452610,11.243532


Completed training for RidgeRegression. CV_RMSE: 20.4526
Training (Fitting) LassoRegression
Fitting 30 folds for each of 4 candidates, totalling 120 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lasso__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,split23_test_score,split24_test_score,split25_test_score,split26_test_score,split27_test_score,split28_test_score,split29_test_score,mean_test_score,std_test_score,rank_test_score
3,0.083607,0.029638,0.021266,0.014522,1.000,{'lasso__alpha': 1.0},-16.672147,-15.872021,-7.224639,-12.888698,...,-5.366948,-10.031750,-30.951034,-38.121383,-40.577025,-25.646150,-36.308846,-20.552705,11.055260,4
0,0.171718,0.067189,0.030657,0.033360,0.001,{'lasso__alpha': 0.001},-15.542118,-15.177760,-6.954014,-12.679420,...,-5.505533,-8.946875,-30.789488,-37.315228,-40.248267,-25.770209,-36.471326,-20.452515,11.244617,3
1,0.135287,0.051807,0.031392,0.033482,0.010,{'lasso__alpha': 0.01},-15.561852,-15.175785,-6.910305,-12.665614,...,-5.474834,-8.964108,-30.816666,-37.346465,-40.291349,-25.771440,-36.458430,-20.450691,11.253308,2
2,0.147227,0.063047,0.032337,0.021619,0.100,{'lasso__alpha': 0.1},-15.680324,-15.225521,-6.756993,-12.681798,...,-5.260677,-9.196576,-30.856528,-37.499363,-40.385222,-25.781598,-36.421182,-20.412825,11.254827,1


...

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gbr__learning_rate,param_gbr__max_depth,param_gbr__n_estimators,params,split0_test_score,split1_test_score,...,split23_test_score,split24_test_score,split25_test_score,split26_test_score,split27_test_score,split28_test_score,split29_test_score,mean_test_score,std_test_score,rank_test_score
11,10.514096,1.152641,0.008669,0.010264,0.10,4,200,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-18.190295,-15.850778,...,-7.032744,-8.665663,-24.926106,-38.271667,-34.805285,-20.008026,-35.560377,-21.102517,11.277413,12
10,5.039531,0.343983,0.008454,0.007015,0.10,4,100,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-16.955280,-18.611598,...,-6.109535,-8.257078,-25.130933,-38.379613,-35.730191,-22.945161,-35.830970,-20.836099,11.283366,11
9,7.564462,0.211912,0.012614,0.017838,0.10,3,200,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-16.910117,-14.986692,...,-6.485563,-7.435284,-25.710513,-41.875279,-36.975384,-23.381570,-36.842715,-20.489620,11.924490,10
5,9.888479,0.402437,0.010758,0.012527,0.05,4,200,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-16.388577,-16.737963,...,-6.331944,-7.803013,-25.320839,-39.169866,-36.002606,-24.751289,-37.142943,-20.403642,11.469921,9
8,3.687694,0.146628,0.006838,0.006059,0.10,3,100,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-17.063327,-15.300741,...,-6.272809,-7.461097,-26.079325,-40.448112,-37.312771,-24.926177,-37.574109,-20.327386,11.762638,8
4,5.454232,0.696053,0.009005,0.009971,0.05,4,100,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-16.450948,-15.450606,...,-6.140852,-8.129032,-25.923066,-37.670757,-36.518643,-25.259401,-38.793438,-20.274853,11.597142,7
3,7.534541,0.451082,0.006807,0.004031,0.05,3,200,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-17.138171,-16.563671,...,-6.170956,-7.516750,-25.952124,-38.433211,-36.985932,-24.584535,-37.917377,-20.240959,11.536216,6
0,2.557020,0.171104,0.008052,0.008761,0.05,2,100,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-18.183123,-17.075518,...,-5.832822,-7.953272,-27.759677,-36.086347,-38.724130,-25.419993,-37.154577,-20.122704,11.424695,5
2,3.798427,0.243311,0.009403,0.009856,0.05,3,100,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-17.098050,-16.415146,...,-6.185923,-7.766032,-26.249750,-35.957870,-37.622104,-25.825765,-38.907321,-20.105057,11.487641,4
7,5.213106,0.244131,0.009272,0.010890,0.10,2,200,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-17.731757,-17.011422,...,-6.371574,-7.460525,-26.602610,-36.697482,-37.579030,-24.667135,-37.245793,-20.086105,11.656033,3


,params,mean_test_score,std_test_score
11,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-21.102517,11.277413
10,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-20.836099,11.283366
9,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-20.489620,11.924490
5,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-20.403642,11.469921
8,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-20.327386,11.762638
4,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-20.274853,11.597142
3,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-20.240959,11.536216
0,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-20.122704,11.424695
2,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-20.105057,11.487641
7,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-20.086105,11.656033


Completed training for GradientBoostingRegressor. CV_RMSE: 19.8605
--------------------------------------------------
Printing the Model Leaderboard sorted by CV_RMSE:


,Model,CV_RMSE,Best Params
5,RandomForestRegressor,19.4867,"{'rfr__max_depth': None, 'rfr__max_features': ..."
6,GradientBoostingRegressor,19.8605,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ..."
4,BaggingRegressor,20.0463,"{'br__max_samples': 0.6, 'br__n_estimators': 100}"
2,LassoRegression,20.4128,{'lasso__alpha': 0.1}
0,LinearRegression,20.4526,Default
1,RidgeRegression,20.4526,{'ridge__alpha': 0.01}
3,KNN,20.4826,"{'knn__n_neighbors': 9, 'knn__weights': 'unifo..."


Saving models to disk...
Done.


## Predicting and Calculating Evaluation Metrics (RMSE and $R^2$)

In [28]:
from sklearn.metrics import root_mean_squared_error as rmse, r2_score
import pandas as pd

# Retrieving estimators dictionary
print("Loading models from disk...")
estimators = joblib.load("../data/processed/estimators.pkl")

# Models
rf_model = estimators["RandomForestRegressor"]
gbr_model = estimators["GradientBoostingRegressor"]

# CV RMSE from leaderboard
rf_cv_rmse = leaderb_df.loc[
    leaderb_df["Model"] == "RandomForestRegressor", "CV_RMSE"
].values[0]

gbr_cv_rmse = leaderb_df.loc[
    leaderb_df["Model"] == "GradientBoostingRegressor", "CV_RMSE"
].values[0]

# Predictions
rf_pred = rf_model.predict(X_test)
gbr_pred = gbr_model.predict(X_test)

# Test metrics
rf_rmse = round(rmse(y_test, rf_pred), 4)
gbr_rmse = round(rmse(y_test, gbr_pred), 4)

rf_r2 = round(r2_score(y_test, rf_pred), 4)
gbr_r2 = round(r2_score(y_test, gbr_pred), 4)

# Generalization gap (absolute difference)
rf_gap = round(abs(rf_rmse - rf_cv_rmse), 4)
gbr_gap = round(abs(gbr_rmse - gbr_cv_rmse), 4)

# Dynamic best vs comparison (lower RMSE is better)
if rf_rmse < gbr_rmse:
    rows = [
        ["RandomForestRegressor", "Best Model", rf_cv_rmse, rf_rmse, rf_r2, rf_gap],
        ["GradientBoostingRegressor", "Comparison Model", gbr_cv_rmse, gbr_rmse, gbr_r2, gbr_gap]
    ]
else:
    rows = [
        ["GradientBoostingRegressor", "Best Model", gbr_cv_rmse, gbr_rmse, gbr_r2, gbr_gap],
        ["RandomForestRegressor", "Comparison Model", rf_cv_rmse, rf_rmse, rf_r2, rf_gap]
    ]

# Final DataFrame
final_results = pd.DataFrame(
    rows,
    columns=["Model", "Type", "CV_RMSE", "Test_RMSE", "Test_R2", "Gap"]
)

display(leaderb_df)
display(final_results)

Loading models from disk...


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.1s finished


,Model,CV_RMSE,Best Params
5,RandomForestRegressor,19.4867,"{'rfr__max_depth': None, 'rfr__max_features': ..."
6,GradientBoostingRegressor,19.8605,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ..."
4,BaggingRegressor,20.0463,"{'br__max_samples': 0.6, 'br__n_estimators': 100}"
2,LassoRegression,20.4128,{'lasso__alpha': 0.1}
0,LinearRegression,20.4526,Default
1,RidgeRegression,20.4526,{'ridge__alpha': 0.01}
3,KNN,20.4826,"{'knn__n_neighbors': 9, 'knn__weights': 'unifo..."


,Model,Type,CV_RMSE,Test_RMSE,Test_R2,Gap
0,GradientBoostingRegressor,Best Model,19.8605,17.1221,0.4010,2.7384
1,RandomForestRegressor,Comparison Model,19.4867,17.8550,0.3486,1.6317


## CONCLUSIONS AND RECOMMENDATIONS